In [1]:
import pandas as pd
from icecream import ic
import numpy as np
import random
from itertools import combinations
from geopy.distance import geodesic
#from itertools import iterrows

cities= pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
cities
ic("454")

ic| '454'


'454'

In [2]:

DIST_MATRIX= np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


for c1 in range(len(cities)):
    DIST_MATRIX[c1,c1]=np.inf


masked_dist_matrix = np.ma.masked_equal(DIST_MATRIX, np.inf)
min_index = np.ma.argmin(masked_dist_matrix[0])

ic(min_index)
#np.argmin(DIST_MATRIX[0])

cities.head()


ic| min_index: np.int64(7)


,name,lat,lon
0,Isangel,-19.53,169.28
1,Lakatoro,-16.09,167.40
2,Longana,-15.30,168.00
3,Luganville,-15.51,167.15
4,Norsup,-16.07,167.39


In [3]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [4]:
def startgreedy(start):
   giro=[]
   costi=[]
   startpoint=globalstart=posto= start #np.int64(np.random.randint(len(cities)))
   giro.append(startpoint)
   copydist=DIST_MATRIX.copy()
   distances = DIST_MATRIX.copy()
   distances[:, posto] = np.inf
      
   for i in range(len(cities)):
      if(i==(len(cities))-1):
         posto=globalstart
      else:  
         posto=np.argmin(distances[startpoint])

      costo=copydist[posto][startpoint]
      costi.append(costo)

      giro.append(posto)
      distances[:, posto] = np.inf
      startpoint=posto

      
      

   #ic(giro,costi)
   return(tsp_cost(giro))






In [ ]:
def startExploring(startpoint,distances,near,giro,ended):
   rightindex= 0
   leftindex=1
   if ended==True:
      rightindex+=2
      leftindex+=2
   near[rightindex]=np.ma.argmin(distances[startpoint])
   distances[startpoint,near[rightindex]] = np.ma.masked
   distances[near[rightindex],startpoint] = np.ma.masked
   
   near[leftindex]=np.ma.argmin(distances[startpoint])
   distance1=distances[startpoint,near[leftindex]]
   distance2=distances[near[rightindex],near[leftindex]]
   if(distance2<distance1):  #link start to 0 and 0 to  1
      distances[near[rightindex]] = np.ma.masked
      distances[:,near[rightindex]] = np.ma.masked
      distances[near[rightindex],near[leftindex]] = np.ma.masked
      distances[near[leftindex],near[rightindex]] = np.ma.masked
      giro[startpoint]=near[rightindex]
      giro[near[rightindex]]=near[leftindex]
      near[rightindex]=near[leftindex]
      near[leftindex]=startpoint
   
   else:  #link start to 0 and start to 1
      distances[startpoint] = np.ma.masked
      distances[:,startpoint] = np.ma.masked
      distances[startpoint,near[leftindex]] = np.ma.masked
      distances[near[leftindex],startpoint] = np.ma.masked
      giro[near[leftindex]]=startpoint
      giro[startpoint]=near[rightindex]


def findNearToNear(near,distances):

   neartonear=near.copy()
   distances[near[0],near[1]] = distances[near[1],near[0]]= np.ma.masked
   neartonear[0]=np.ma.argmin(distances[near[0]])
   distances[near[1],neartonear[0]] =  distances[neartonear[0],near[1]]=np.ma.masked
   neartonear[1]=np.ma.argmin(distances[near[1]])
   distances[near[2],near[3]] = distances[near[3],near[2]] = np.ma.masked
   neartonear[2]=np.ma.argmin(distances[near[2]])
   distances[near[3],neartonear[2]] =distances[neartonear[2],near[3]] =  np.ma.masked
   neartonear[3]=np.ma.argmin(distances[near[3]])







def startgreedy2(start):
   giro=[-1] * len(cities)
   n_link= np.zeros(len(cities))
   girovec= np.zeros(len(cities))



   costi=[]
   startpoint=globalstart=posto= start #np.int64(np.random.randint(len(cities)))
   
   
   copydist=DIST_MATRIX.copy()
   distances = np.ma.masked_equal(DIST_MATRIX, np.inf)
   near=[0,0,0,0]


   end=np.ma.argmax(distances[startpoint])


   startExploring(startpoint,distances,near,giro,False)
   startExploring(end,distances,near,giro,True)

   neartonear=near.copy()

   
   
   distances[near[0],near[1]] = np.ma.masked
   distances[near[1],near[0]] = np.ma.masked

   neartonear[0]=np.ma.argmin(distances[near[0]])
   distances[near[1],neartonear[0]] = np.ma.masked
   distances[neartonear[0],near[1]] = np.ma.masked
   neartonear[1]=np.ma.argmin(distances[near[1]])
   distances.mask[near[0],near[1]] = False
   distances.mask[near[1],near[0]] = False


   distances[near[2],near[3]] = np.ma.masked
   distances[near[3],near[2]] = np.ma.masked

   neartonear[2]=np.ma.argmin(distances[near[2]])
   distances[near[3],neartonear[2]] = np.ma.masked
   distances[neartonear[2],near[3]] = np.ma.masked
   neartonear[3]=np.ma.argmin(distances[near[3]])
   distances.mask[near[3],near[2]] = False
   distances.mask[near[2],near[3]] = False
   ic(neartonear)
   ic(near)

   

   """
   near[0]=np.ma.argmin(distances[startpoint])
   distances[startpoint,near[0]] = np.ma.masked
   distances[near[0],startpoint] = np.ma.masked
   
   near[1]=np.ma.argmin(distances[startpoint])
   distance1=distances[startpoint,near[1]]
   distance2=distances[near[0],near[1]]
   if(distance2<distance1):  #link start to 0 and 0 to  1
      distances[near[0]] = np.ma.masked
      distances[:,near[0]] = np.ma.masked
      distances[near[0],near[1]] = np.ma.masked
      distances[near[1],near[0]] = np.ma.masked
      giro[startpoint]=near[0]
      giro[near[0]]=near[1]
      near[0]=near[1]
      near[1]=startpoint
   
   else:  #link start to 0 and start to 1
      distances[startpoint] = np.ma.masked
      distances[:,startpoint] = np.ma.masked
      distances[startpoint,near[1]] = np.ma.masked
      distances[near[1],startpoint] = np.ma.masked
      giro[near[1]]=startpoint
      giro[startpoint]=near[0]
   
   
   """
   ic(giro)





   
  

   #distances[:, posto] = np.inf





   
   """
   for i in range(len(cities)):
      if(i==(len(cities))-1):
         posto=globalstart
      else:  
         posto=np.argmin(distances[startpoint])
      
      n_link[posto]+=1
      n_link[startpoint]+=1
      


      costo=copydist[posto][startpoint]
      costi.append(costo)

      giro.append(posto)
      distances[:, posto] = np.inf
      startpoint=posto

      
      

   
   ic(giro)
   return(tsp_cost(giro))
   """  

   return


startgreedy2(0)

ic| neartonear: [np.int64(4), np.int64(2), np.int64(3), np.int64(4)]
ic| near: [np.int64(1), 0, np.int64(2), np.int64(6)]
ic| giro: [np.int64(7), -1, -1, -1, -1, np.int64(2), np.int64(5), np.int64(1)]


In [159]:
np.argmin(DIST_MATRIX[0])

np.int64(7)

In [160]:
totalcost=0

for i in range(1):
    totalcost+=startgreedy2(i)
    #ic(startgreedy(i))
    #ic(totalcost)


ic| giro: [0,
           np.int64(7),
           np.int64(1),
           np.int64(4),
           np.int64(3),
           np.int64(5),
           np.int64(2),
           np.int64(6),
           0]
